In [3]:
# from tkinter import *
# from tkinter import filedialog
# from PIL import Image, ImageTk

# def resize_image(event):
#     # Resize the image to fit the canvas dimensions
#     new_width = event.width
#     new_height = event.height
#     image = original_image.resize((new_width, new_height), Image.ANTIALIAS)
#     new_img = ImageTk.PhotoImage(image)
#     canvas.config(image=new_img)
#     canvas.image = new_img

# if __name__ == "__main__":
#     root = Tk()

#     # Set the initial window size using the geometry method
#     root.geometry("1800x1600")  # Adjust the size as needed

#     frame = Frame(root, bd=2, relief=SUNKEN)
#     frame.grid_rowconfigure(0, weight=1)
#     frame.grid_columnconfigure(0, weight=1)

#     xscroll = Scrollbar(frame, orient=HORIZONTAL)
#     xscroll.grid(row=1, column=0, sticky=E+W)
#     yscroll = Scrollbar(frame)
#     yscroll.grid(row=0, column=1, sticky=N+S)

#     canvas = Canvas(frame, bd=0, xscrollcommand=xscroll.set, yscrollcommand=yscroll.set)
#     canvas.grid(row=0, column=0, sticky=N+S+E+W)

#     xscroll.config(command=canvas.xview)
#     yscroll.config(command=canvas.yview)
#     frame.pack(fill=BOTH, expand=1)

#     full_path = filedialog.askopenfilename(parent=root, initialdir="D:/senior 2 term 1/image processing/draw-on-image/images-to-be-tested", title='Choose an image.')
#     original_image = Image.open(full_path)
#     img = ImageTk.PhotoImage(original_image.resize((800, 600), Image.ANTIALIAS))  # Resize image initially
#     canvas.create_image(0, 0, image=img, anchor="nw")

#     canvas.config(scrollregion=canvas.bbox(ALL))
#     canvas.bind("<Configure>", resize_image)

#     def printcoords(event):
#         print(event.x, event.y)

#     canvas.bind("<Button 1>", printcoords)

#     root.mainloop()


In [4]:
from tkinter import *
from tkinter import filedialog
from PIL import Image, ImageTk, ImageDraw
import os

def display_image(image, title="Image"):
    # Display the image in a new window
    display_window = Toplevel(root)
    display_window.title(title)

    # Convert the PIL Image to a Tkinter PhotoImage
    img_tk = ImageTk.PhotoImage(image)

    # Create a label to display the image
    label = Label(display_window, image=img_tk)
    label.image = img_tk
    label.pack()
def draw_line(event):
    global line_start
    if line_start is None:
        line_start = (event.x, event.y)
    else:
        line_end = (event.x, event.y)

        # Adjust coordinates based on the canvas view
        x0, y0 = canvas.canvasx(line_start[0]), canvas.canvasy(line_start[1])
        x1, y1 = canvas.canvasx(line_end[0]), canvas.canvasy(line_end[1])

        canvas.create_line(x0, y0, x1, y0, fill="black", width=20)
        #line 2

        canvas.create_line(x0, y0, x0, y1, fill="black", width=20)

         #line 3

        canvas.create_line(x1, y1, x1, y0, fill="black", width=20)

        #line 4

        canvas.create_line(x1, y1, x0, y1, fill="black", width=20)

        lines.append((x0, y0, x1, y0))

        lines.append((x0, y0, x0, y1))
        lines.append((x1, y1, x1, y0))
        lines.append((x1, y1, x0, y1))
        line_start=None

def save_image():
    global original_image  # Ensure original_image is a global variable

    # Create a drawing object
    draw = ImageDraw.Draw(original_image)

    # Draw lines on the image
    for line in lines:
        draw.line(line, fill="black", width=12)

    # Display the image before drawing
    display_image(original_image, title="Before Drawing")

    # Set the initial directory to the desktop
    desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")

    # Prompt the user to choose a file location to save the image
    file_path = filedialog.asksaveasfilename(defaultextension=".png", filetypes=[("PNG files", "*.png"), ("All files", "*.*")], initialdir=desktop_path)

    # Save the image to the chosen file location
    if file_path:
        original_image.save(file_path)

def on_canvas_click(event):
    global last_x, last_y
    last_x, last_y = event.x, event.y

def on_canvas_drag(event):
    global last_x, last_y
    x, y = event.x, event.y
    canvas.scan_dragto(last_x - x, last_y - y, gain=1)
    last_x, last_y = x, y

def on_canvas_zoom(event):
    global scale_factor
    if event.keysym == "q":
        canvas.scale("all", event.x, event.y, scale_factor, scale_factor)
    elif event.keysym == "e":
        canvas.scale("all", event.x, event.y, 1/scale_factor, 1/scale_factor)


if __name__ == "__main__":
    root = Tk()

    # Set the initial window size using the geometry method
    root.geometry("800x600")  # Adjust the size as needed

    frame = Frame(root, bd=2, relief=SUNKEN)
    frame.grid_rowconfigure(0, weight=1)
    frame.grid_columnconfigure(0, weight=1)

    xscroll = Scrollbar(frame, orient=HORIZONTAL)
    xscroll.grid(row=1, column=0, sticky=E+W)
    yscroll = Scrollbar(frame)
    yscroll.grid(row=0, column=1, sticky=N+S)

    canvas = Canvas(frame, bd=0, xscrollcommand=xscroll.set, yscrollcommand=yscroll.set)
    canvas.grid(row=0, column=0, sticky=N+S+E+W)

    xscroll.config(command=canvas.xview)
    yscroll.config(command=canvas.yview)
    frame.pack(fill=BOTH, expand=1)

    full_path = filedialog.askopenfilename(parent=root, initialdir=".", title='Choose an image.')
    original_image = Image.open(full_path)
    img = ImageTk.PhotoImage(original_image)  # No resizing initially
    canvas.create_image(0, 0, image=img, anchor="nw", tags="all")

    canvas.configure(scrollregion=canvas.bbox("all"))

    line_start = None
    lines = []
    last_x, last_y = 0, 0


    canvas.bind("<Button 1>", draw_line)
    canvas.bind("<Key>", on_canvas_zoom)

    save_button = Button(root, text="Save Image", command=save_image)
    save_button.pack(side=BOTTOM)

    root.mainloop()


C:\Users\admin\AppData\Local\Temp\ipykernel_21180\2498516742.py:129: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  resized_image = original_image.resize((new_width, new_height), Image.ANTIALIAS)


AttributeError: 'Canvas' object has no attribute 'c'

: 